### Single and Multi Headed attention

- Query, key, value vectors are not identical in practice.
- They are each a linear transformation of embedding vector.
- This allows self attention layer to focus on complementary words in the context rather than the same word itself.
- Linear transformation is done with learnable parameters.
- In single headed attn, only transformation is created for each of query, key and value
- In multi headed attn, multiple transformations are created.
- Each head encode a different aspect of similarity like subject-verb interaction, nearby adjectives etc.
- These as aspects are fully learned from data and are similar to filters in CNNs.


Single headed attention


In [42]:
from transformers import AutoTokenizer, AutoConfig, AutoModel
from bertviz import head_view

from torch import nn, bmm
import torch
import numpy as np
from math import sqrt

In [11]:
def scaled_dot_prod_attn(query, key, value):

    dim_k = query.size(-1)
    scores = bmm(query, key.transpose(1,2))/sqrt(dim_k)
    weights = nn.functional.softmax(scores, dim=-1)
    attn_outputs = bmm(weights, value)
    return attn_outputs

One attention head

embed_dim:

head_dim:

In [17]:
class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim) -> None:
        super().__init__()
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)

    def forward(self, hidden_state):
        attn_outputs = scaled_dot_prod_attn(self.q(hidden_state),
                                                    self.k(hidden_state),
                                                    self.v(hidden_state))
        return attn_outputs

Multi attention heads

In [34]:
class MultiAttentionHead(nn.Module):
    def __init__(self, config) -> None:
        super().__init__()
        embed_dim = config.hidden_size
        num_heads = config.num_attention_heads
        head_dim = embed_dim//num_heads
        self.heads = nn.ModuleList([AttentionHead(embed_dim, head_dim) for _ in range(num_heads)])
        self.output_linear = nn.Linear(embed_dim, embed_dim)

    def forward(self, hidden_state):
        x = torch.cat([h(hidden_state) for h in self.heads], dim=-1)
        x = self.output_linear(x)
        return x
        


In [35]:
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

text = 'Its better to take action than wonder'
inputs = tokenizer(text, return_tensors='pt', add_special_tokens=False)

config = AutoConfig.from_pretrained(model_name)
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
token_emb

input_embeds = token_emb(inputs.input_ids)

In [36]:
config.vocab_size, config.hidden_size

(30522, 768)

In [37]:
config.num_attention_heads

12

In [38]:
768//12

64

In [39]:
input_embeds.size()

torch.Size([1, 7, 768])

In [40]:
multihead_attn = MultiAttentionHead(config)
attn_output = multihead_attn(input_embeds)
attn_output.size()

torch.Size([1, 7, 768])

In [43]:
model = AutoModel.from_pretrained(model_name, output_attentions=True)

sentence_a = "I just can't put this novel down"
sentence_b = "Language models are a novel invention"

viz_inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt')
attention = model(**viz_inputs).attentions
sentence_b_start = (viz_inputs.token_type_ids == 0).sum(dim=1)
tokens = tokenizer.convert_ids_to_tokens(viz_inputs.input_ids[0])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [46]:
head_view(attention, tokens, sentence_b_start, heads=[8])

<IPython.core.display.Javascript object>

Save attention output

In [50]:
torch.save(attn_output, 'attn_output.pt')